<a href="https://colab.research.google.com/github/neennera/GIBBIN-Web-Scraping/blob/main/WebScaping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---



# [FIXED] open google chrom by selenium
support ubuntu 20.04 due to colab update ubanto version

In [50]:
destination = "https://esc.kmutt.ac.th/index.php"

In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

In [72]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")

options.headless = True

driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)

driver.get(destination)
print(driver.title)

<ipython-input-72-e12a53def73d>:8: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True
<ipython-input-72-e12a53def73d>:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)


ESC - ESC




---

# Web Scraping : Go to 'Student' page
- Get HTML source code using BeautifulSoup
- Read and transfer code to lecture data using HTMLParser

In [73]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


X_path_login_username = '//html/body/div[2]/div[1]/div/div[2]/div/div/div/form/div/div[1]/div[2]/div/input'
X_path_login_password = '//html/body/div[2]/div[1]/div/div[2]/div/div/div/form/div/div[2]/div[2]/div/input'
X_path_login_submitButton = '//html/body/div[2]/div[1]/div/div[2]/div/div/div/form/div/div[6]/div/input'

#username_input = driver.find_element("id","username")
username_input = driver.find_element("xpath",X_path_login_username)
username_input.send_keys(gibbon_username)

password_input = driver.find_element("xpath",X_path_login_password)
password_input.send_keys(gibbon_password)

login_box = driver.find_element("xpath",X_path_login_submitButton)
login_box.submit()

In [75]:
Xpath_of_learn_button = '//*[@id="header-menu"]/ul/li[3]'
Xpath_of_timetable_button = '//*[@id="header-menu"]/ul/li[3]/ul/li[4]/a'

driver.find_element('xpath',Xpath_of_learn_button)
timetable_page = driver.find_element('xpath',Xpath_of_timetable_button).get_property("href")

In [76]:
driver.get(timetable_page)

# Web Scraping : Read the class timetable

In [ ]:
ESCK_Class = ['4E','4S','4C','5E','5S','5C','6E','6S','6C','AU1/1','AU1/2','BI1/1','BI1/2','AU2/1','AU2/2','AU3/1']

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [133]:
from html.parser import HTMLParser

class MyHTMLParser(HTMLParser):
  def __init__(self):
    HTMLParser.__init__(self)
    HTMLParser.unwanted_tag = ['div','i','a','br']
    HTMLParser.lectuteInWeek = []
    HTMLParser.day = 0

  def GetLectuteInWeek(self, HTMLtext):
    for i in range(0,7):
      HTMLParser.lectuteInWeek.append([])
      parser.feed(HTMLtext[i])
      HTMLParser.day += 1
    return HTMLParser.lectuteInWeek
  
  def handle_data(self, data):
    # print("Encountered some data  :", data)
    HTMLParser.lectuteInWeek[HTMLParser.day].append(data)

  # def handle_starttag(self, tag, attrs):
  #   if not(tag in HTMLParser.unwanted_tag) :
  #     print("Encountered a start tag:", tag)

  # def handle_endtag(self, tag):
  #   if not(tag in HTMLParser.unwanted_tag) :
  #     print("Encountered an end tag :", tag)

In [89]:
parser = MyHTMLParser()

In [135]:
Xpath_of_search_box = '//*[@id="search"]'
Xpath_of_search_submit_button = '//*[@id="ttView"]/div/div[2]/div/input'
Xpath_of_View_detail_button = '//*[@id="timetables"]/table/tbody/tr[1]/td[5]/nav/div/a'

def searching(classroom) :
  driver.get(timetable_page)
  driver.find_element('xpath',Xpath_of_search_box).clear()
  driver.find_element('xpath',Xpath_of_search_box).send_keys(classroom)
  driver.find_element('xpath',Xpath_of_search_submit_button).submit()
  driver.find_element('xpath',Xpath_of_View_detail_button).click()

def scaping():
  lecture = []
  for i in range(0,7):
    get_div = driver.find_element('xpath','//*[@id="ttWrapper"]/table/tbody/tr[2]/td[' + str(i+2) + ']')
    lecture.append(get_div.get_attribute('innerHTML'))
  parser = MyHTMLParser()
  lectureInWeek = parser.GetLectuteInWeek(lecture)
  return lectureInWeek

In [ ]:
driver.page_source

In [ ]:
#searching("6C")
exampleOfLecture = scaping()

In [ ]:
# driver.get(destination)
# soup =  BeautifulSoup(driver.page_source, 'html.parser')

# Analyse Data

In [136]:
temp = exampleOfLecture.copy()

In [115]:
class lecture:
  def __init__(self, day=-1 , start=-1, end=-1, subject="none", className="none"):
    self.day = day
    self.start = start
    self.end = end
    self.className = className
    self.subject = subject

In [116]:
lectureData = []

In [137]:
for day in range(0,7):
  temp = exampleOfLecture[day].copy()
  i=0
  while i< len(temp): 
    if temp[i][0:4] == "ESC ":  
      
      #get subject and class name
      subject = temp[i][0:3] + temp[i][4:-2]
      if subject[-1] == ".":
        subject = subject[:-1]
      className = temp[i+1]

      # get time
      timeIndex = temp.index(subject) +1
      time = temp[timeIndex]
      start = time[0:5]
      end = time[-5:]
      
      # remove read class
      temp.remove(subject)
      temp.remove(time)

      #append data
      print(str(day) + "  " + subject + " ; " + className + " ; ", start + " " + end)
      lectureData.append(lecture(day,start,end,subject,className))
      continue

    if temp[i] == "School Closed":
      print(str(day) + "  " + "School Closed")
      lectureData.append(lecture(day, "08:00" , "16:00", "School Closed"))
      break
    
    i+=1

1  ESC623 ; Disscution of problem ;  09:00 11:00
1  ESC625 ; ภาษาถิ่น (ภาษาและวัฒนธรรม) ;  12:00 14:00
1  ESC652 ; Unit 10B: The Power of Play ;  14:00 15:00
1  ESC642 ; ทักษะการสกัดกั้น(บล็อก) ;  15:00 16:00
2  ESC624 ; Endocrine system ;  09:00 11:00
2  ESC625 ; ภาษาถิ่น (กิจกรรม) ;  12:00 14:00
2  ESC652 ; Unit 10B: Negative Adverb, Unit 10C: Living Free? ;  14:00 16:00
3  ESC623 ; สื่อโฆษณาเพื่อสุขภาพ 2 ;  10:00 11:00
3  ESC728 ; review and midterm examination ;  12:00 14:00
3  ESC711 ; Calculus with parametric curves ;  14:00 16:00
4  ESC623 ; การทำ Storyboard ;  09:00 10:00
4  ESC713 ; การประยุกต์ของสมการเชิงเส้นอันดับหนึ่ง (1) ;  10:00 12:00
4  ESC721 ; Oscillatory motion I ;  13:00 15:00
4  ESC623 ; (EXT) Self-study ;  16:00 18:00
5  ESC621 ; แผนผังคาร์โนห์ 1 ;  09:00 11:00
5  ESC621 ; พีชคณิตบูลีนและการลดรูปสมการลอจิก ;  12:00 14:00
5  ESC623 ; Solve the problem ;  14:00 16:00
